In [16]:
import torch
#import torchvision
#import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import KddModel as kdd

In [17]:
batch_size = 100
use_gpu=torch.cuda.is_available()
root = '../../data/kdd/'
transform = torch.FloatTensor

# dataset
trainset = kdd.Nslset("KDDTrain+.txt", train=True, transform=transform)
testset = kdd.Nslset("KDDTest+.txt", train=False, transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

# constant for classes
classes = ('normaly', 'anormal')

In [15]:
print(trainset.size())

AttributeError: 'Nslset' object has no attribute 'size'

In [3]:
# CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convmodel = nn.Sequential(nn.Conv2d(1, 6, 3, padding=2),
                                       nn.BatchNorm2d(6),
                                       nn.Sigmoid(),
                                       nn.Conv2d(6, 16, 3),
                                       nn.BatchNorm2d(16),
                                       nn.Sigmoid(),
                                       )
        self.fcmodel = nn.Sequential(nn.Linear(16*7*7, 200),
                                     nn.BatchNorm1d(200),
                                     nn.Sigmoid(),
                                     nn.Linear(200, 100),
                                     nn.Sigmoid(),
                                     nn.Linear(100, 20),
                                     nn.Sigmoid(),
                                     nn.Linear(20, 2)
                                     )

    def forward(self, out):
        out = self.convmodel(out)
        out = out.view(-1, 16*7*7)
        out = self.fcmodel(out)
        return out

In [4]:
def accuracy(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            cvdata, labels = data
            if(use_gpu):
                cvdata=cvdata.cuda()
                labels=labels.cuda()
            outputs = net(cvdata)
            _, predicted = torch.max(outputs.data, 1)
            predicted[predicted > 0] = 1.
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    #print("correct:%d  total:%d" %(correct,total))
    print('Accuracy: %d %%' % (100 * correct / total))

In [5]:
net = Net()
criterion = nn.CrossEntropyLoss()
if(use_gpu):
    net=net.cuda()
    criterion=criterion.cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, 6,gamma=0.1)

In [6]:
running_loss = 0.0
mini_batches_count = 1000
epochs=20
for epoch in range(epochs):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if(use_gpu):
            inputs=inputs.cuda()
            labels=labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # whatur(labels)
        # break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # every 1000 mini-batches...
        if i % mini_batches_count == (mini_batches_count-1):
            # scheduler.step()
            print("[%d,%6d] ava_loss: %.5f" %
                  (epoch+1, i+1, running_loss/mini_batches_count))
            running_loss = 0.0

    accuracy(net, testloader)
print('Finished Training')

[1,  1000] ava_loss: 0.15622
Accuracy: 78 %
[2,  1000] ava_loss: 0.08823
Accuracy: 81 %
[3,  1000] ava_loss: 0.06143
Accuracy: 80 %
[4,  1000] ava_loss: 0.04890
Accuracy: 77 %
[5,  1000] ava_loss: 0.04639
Accuracy: 84 %
[6,  1000] ava_loss: 0.03688
Accuracy: 83 %
[7,  1000] ava_loss: 0.03241
Accuracy: 80 %
[8,  1000] ava_loss: 0.03307
Accuracy: 79 %
[9,  1000] ava_loss: 0.02710
Accuracy: 84 %
[10,  1000] ava_loss: 0.02560
Accuracy: 82 %
[11,  1000] ava_loss: 0.02521
Accuracy: 81 %
[12,  1000] ava_loss: 0.02249
Accuracy: 78 %
[13,  1000] ava_loss: 0.02323
Accuracy: 81 %
[14,  1000] ava_loss: 0.02179
Accuracy: 81 %
[15,  1000] ava_loss: 0.02166
Accuracy: 81 %
[16,  1000] ava_loss: 0.01970
Accuracy: 82 %
[17,  1000] ava_loss: 0.01979
Accuracy: 80 %
[18,  1000] ava_loss: 0.02031
Accuracy: 82 %
[19,  1000] ava_loss: 0.01892
Accuracy: 79 %
[20,  1000] ava_loss: 0.01748
Accuracy: 79 %
Finished Training


In [7]:
PATH = "../../pth/nsl_kdd_test.pth"
torch.save(net.state_dict(), PATH)

In [8]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [9]:
class_correct = list(0 for i in range(2))
class_total = list(0 for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        predicted[predicted > 0] = 1
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('total:%d,correct:%d' % (class_total[i], class_correct[i]))
    print('Accuracy: %.4f %%' % (100 * class_correct[i] / class_total[i]))

total:9711,correct:9396
Accuracy: 96.7563 %
total:12833,correct:8576
Accuracy: 66.8277 %
